In [13]:
import requests
import pandas as pd
import os

In [23]:
def fetch_all_data(params=None, max_pages=None):
    """
    Fetch all pages of data from a paginated API and build DataFrame incrementally

    Parameters:
    base_url: API endpoint URL
    params: Dictionary of query parameters
    max_pages: Optional limit on number of pages to fetch

    Returns:
    Pandas DataFrame with all results
    """
    df = pd.DataFrame()
    page = 1
    params = params or {}
    subdomain = os.getenv('MAXIO_SUBDOMAIN')
    base_url = f"https://{subdomain}.chargify.com/events.json"
    username = os.getenv('MAXIO_API_KEY')
    password = 'x'
    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }

    while True:
        # Add pagination parameters
        params['page'] = page
        params['per_page'] = 200  # Using max allowed per page

        response = requests.get(base_url, auth=(username, password), params=params, headers=headers)
        response.raise_for_status()  # Raise exception for HTTP errors

        # Parse the JSON response
        data = response.json()

        # Assuming data is returned as a list of records
        # Adjust the key if your API returns data in a different structure
        results = data if isinstance(data, list) else None
        # Break if no more results
        if not results:
            break

        # Normalize the results and append to DataFrame
        page_df = pd.json_normalize(results)
        df = pd.concat([df, page_df], ignore_index=True)

        print(f"Retrieved page {page} with {len(results)} results. Total rows so far: {len(df)}")

        # Break if we've reached max_pages
        if max_pages and page >= max_pages:
            break

        # Increment page counter
        page += 1

    return df

In [ ]:
df = fetch_all_data(
    params={'filter': 'metered_usage'}
)

# Rename columns keeping only the last part of the name flattened from the JSON structure
df.columns = [col.split('.')[-1] for col in df.columns]


Retrieved page 1 with 32 results. Total rows so far: 32
Data saved to events.csv
Data saved to events2.csv


In [29]:
df.columns = [
    'Event ID', 'Event Key', 'Event Message', 'Subscription ID', 'Customer ID',
    'Created At', 'Previous Unit Balance', 'New Unit Balance', 'Usage Quantity',
    'Component ID', 'Component Handle', 'Memo'
]

df = df.astype({
    'Event ID': 'str',
    'Subscription ID': 'str',
    'Customer ID': 'str',
    'Component ID': 'str',
    'Previous Unit Balance': 'float',
    'New Unit Balance': 'float',
    'Created At': 'datetime64[ns, UTC]',
    'Memo': 'str'
})


In [30]:
# pd.set_option('display.float_format', lambda x: '%.3f' % x)
df.describe()


,Previous Unit Balance,New Unit Balance,Usage Quantity
count,32.000000,32.000000,32.000000
mean,233.454063,392.258594,158.750000
std,622.514566,824.619952,484.511527
min,0.000000,0.000000,-535.000000
25%,0.000000,9.105000,5.000000
50%,5.887500,35.310000,8.000000
75%,251.250000,327.750000,60.000000
max,3323.000000,3348.000000,2323.000000


In [17]:
df.to_csv('output.csv', index=False)

In [31]:
(
    df
    .groupby(
        [pd.Grouper(freq='ME', key='Created At'), 'Component Handle', 'Subscription ID']
    )['Usage Quantity']
    .sum()
    .reset_index()
)

,Created At,Component Handle,Subscription ID,Usage Quantity
0,2022-10-31 00:00:00+00:00,minutes,56315975,200
1,2023-05-31 00:00:00+00:00,minutes,63077591,0
2,2023-10-31 00:00:00+00:00,minutes,55711722,5
3,2023-10-31 00:00:00+00:00,minutes,67953313,3348
4,2023-12-31 00:00:00+00:00,minutes,70209296,892
5,2024-04-30 00:00:00+00:00,minutes,73290592,44
6,2024-05-31 00:00:00+00:00,minutes,67953313,12
7,2024-05-31 00:00:00+00:00,minutes,73339943,12
8,2024-07-31 00:00:00+00:00,minutes,76184241,5
9,2025-01-31 00:00:00+00:00,sms-messaging,80535405,10
